### tabularizePatientTable
This notebook extracts all data from the Patient table (`patient`) in the Master Patient Index and tabularizes the data into Lists of Lists of Lists (LoLoL).

In [ ]:
pip install psycopg2-binary azure-identity phdi==1.0.6

In [ ]:
import psycopg2
from psycopg2.sql import Identifier, SQL
from azure.identity import DefaultAzureCredential
from phdi.linkage.postgres import DIBBsConnectorClient

In [ ]:
# Set your Key Vault information
vault_name = "$KEY_VAULT"
KEY_VAULT_URL = f"https://{vault_name}.vault.azure.net"

vault_linked_service = "$KEY_VAULT_LINKED_SERVICE"
credential = DefaultAzureCredential()
db_password =  TokenLibrary.getSecret(vault_name,"mpi-db-password",vault_linked_service)

# Set up db_client
DB_NAME = "DibbsMpiDB"
DB_USER = "postgres"
DB_HOST = "$MPI_DB_HOST"
DB_PORT = "5432"
DB_TABLE_PATIENT = "patient"
DB_TABLE_PERSON= "person"

db_client = DIBBsConnectorClient(database = DB_NAME, user = DB_USER, password = db_password, host= DB_HOST, port = DB_PORT, patient_table= DB_TABLE_PATIENT, person_table=DB_TABLE_PERSON)

# Create a connection and a cursor
conn = db_client.get_connection()
cur = conn.cursor()

# Generate query and data to extract all data from patient table
def generate_query(db_client):
    select_query_stubs = []
    query_data = []
    for key in db_client.fields_to_jsonpaths:
        query = f"jsonb_path_query_array(patient_resource,%s) as {key}"
        select_query_stubs.append(query)
        query_data.append(db_client.fields_to_jsonpaths[key])

    select_query = "SELECT " + ", ".join(stub for stub in select_query_stubs)

    query = select_query + " FROM {patient_table};"
    query = SQL(query).format(patient_table=Identifier(db_client.patient_table))
    return query, query_data

query, query_data = generate_query(db_client)

# Execute query
cur.execute(query, query_data)
data = [list(row) for row in cur.fetchall()]

# Format returned data as Lists of Lists of Lists (LoLoL)
def format_data(data, db_client):
    db_client._close_connections(db_conn=conn, db_cursor=cur)

    data_cols = []
    for key in sorted(list(db_client.fields_to_jsonpaths.keys())):
        data_cols.append(key)
    data.insert(0, data_cols)
    return data

formatted_data = format_data(data,db_client)